In [24]:
!nvidia-smi  # Verifica se GPU está sendo reconhecida
!pip list | grep tensorrt  # Verifica a instalação do TensorRT

Wed Oct 30 19:11:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8               4W /  60W |      9MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
import torch.nn as nn

class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(64 * 111 * 111, 10)  # ajuste os números conforme necessário
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [20]:
!pip install torch torchvision matplotlib numpy
!pip install onnx  # necessário para exportação ONNX


In [21]:
!pip list | grep onnx

onnx                      1.17.0


In [22]:
import torch
print(f"CUDA disponível: {torch.cuda.is_available()}")
print(f"Dispositivo atual: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

CUDA disponível: False
Dispositivo atual: CPU


In [23]:
!pip install nvidia-tensorrt
import sys
import torch
import torchvision

print(f"Python versão: {sys.version}")
print(f"PyTorch versão: {torch.__version__}")
print(f"Torchvision versão: {torchvision.__version__}")

Python versão: 3.12.5 | packaged by conda-forge | (main, Aug  8 2024, 18:36:51) [GCC 12.4.0]
PyTorch versão: 2.4.1+cu121
Torchvision versão: 0.19.1+cpu


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Definir transformações para as imagens
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Carregar o dataset CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                     download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                       shuffle=False, num_workers=2)

# Classes do CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [02:00<00:00, 1411976.32it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Criar o modelo
model = SimpleConvNet()

# Verificar se GPU está disponível
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

/home/anderson/miniconda3/envs/scoras_academy/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


SimpleConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU()
)

In [9]:
def train_model(model, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    print("Iniciando treinamento...")
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 2000 == 1999:
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
    
    print('Treinamento finalizado!')
    
    # Salvar o modelo
    torch.save(model.state_dict(), 'cifar_net.pth')

In [14]:
# Treinar o modelo
train_model(model)

# Exportar para ONNX
model.eval()
dummy_input = torch.randn(1, 3, 32, 32).to(device)
torch.onnx.export(model,
                 dummy_input,
                 "meu_modelo.onnx",
                 export_params=True,
                 opset_version=11,
                 do_constant_folding=True,
                 input_names=['input'],
                 output_names=['output'])

print("Modelo exportado para ONNX com sucesso!")

Iniciando treinamento...
[1,  2000] loss: 0.978
[1,  4000] loss: 0.960
[1,  6000] loss: 1.008
[1,  8000] loss: 0.990
[1, 10000] loss: 1.011
[1, 12000] loss: 1.004
[2,  2000] loss: 0.910
[2,  4000] loss: 0.928
[2,  6000] loss: 0.950
[2,  8000] loss: 0.956
[2, 10000] loss: 0.967
[2, 12000] loss: 0.967
[3,  2000] loss: 0.873
[3,  4000] loss: 0.879
[3,  6000] loss: 0.887
[3,  8000] loss: 0.938
[3, 10000] loss: 0.933
[3, 12000] loss: 0.923
[4,  2000] loss: 0.809
[4,  4000] loss: 0.850
[4,  6000] loss: 0.865
[4,  8000] loss: 0.886
[4, 10000] loss: 0.881
[4, 12000] loss: 0.908
[5,  2000] loss: 0.794
[5,  4000] loss: 0.817
[5,  6000] loss: 0.853
[5,  8000] loss: 0.865
[5, 10000] loss: 0.857
[5, 12000] loss: 0.849
Treinamento finalizado!
Modelo exportado para ONNX com sucesso!


In [15]:
def test_model():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Acurácia na base de teste: {100 * correct / total}%')

# Testar o modelo
test_model()

Acurácia na base de teste: 62.67%


# Análise Detalhada do Notebook: Treinamento de CNN com NVIDIA/CUDA

## 1. Verificação e Setup do Ambiente

### Verificação de Hardware e Dependências
* O notebook começa verificando a disponibilidade de GPU NVIDIA através do comando `nvidia-smi`
* Verifica a instalação do TensorRT (framework de otimização da NVIDIA)
* Instalação das dependências necessárias:
  - PyTorch
  - Torchvision
  - Matplotlib
  - NumPy
  - ONNX (para exportação do modelo)

### Status do Ambiente
* Python versão: 3.12.5
* PyTorch versão: 2.4.1+cu121 (compilado com suporte CUDA 12.1)
* Torchvision versão: 0.19.1+cpu

⚠️ **Observação Importante**: Foi detectado um erro na inicialização do CUDA:
```
CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment
```
Isto indica que, apesar do PyTorch ter sido instalado com suporte a CUDA, o treinamento acabou sendo executado na CPU.

## 2. Arquitetura do Modelo

### SimpleConvNet
Uma Rede Neural Convolucional (CNN) foi implementada com a seguinte arquitetura:

1. **Camadas Convolucionais**:
   * `conv1`: Conv2d(3, 6, kernel_size=5) - Entrada RGB (3 canais) → 6 filtros
   * `conv2`: Conv2d(6, 16, kernel_size=5) - 6 canais → 16 filtros

2. **Camadas Fully Connected**:
   * `fc1`: Linear(400, 120)
   * `fc2`: Linear(120, 84)
   * `fc3`: Linear(84, 10) - Saída para as 10 classes do CIFAR-10

3. **Camadas de Ativação e Pooling**:
   * ReLU como função de ativação
   * MaxPool2d(2, 2) para redução de dimensionalidade

## 3. Dataset e Preparação dos Dados

* Utilização do dataset CIFAR-10:
  - 10 classes diferentes
  - Imagens RGB 32x32 pixels
  - 50.000 imagens para treino
  - 10.000 imagens para teste

* **Transformações aplicadas**:
  - Conversão para tensor
  - Normalização com média (0.5) e desvio padrão (0.5)

* **DataLoader Configuration**:
  - Batch size: 4
  - Shuffle: True (para treino)
  - num_workers: 2 (paralelização do carregamento)

## 4. Processo de Treinamento

### Configurações:
* Otimizador: SGD (Stochastic Gradient Descent)
  - Learning rate: 0.001
  - Momentum: 0.9
* Loss Function: CrossEntropyLoss
* Epochs: 5

### Análise do Treinamento:
* O loss diminuiu consistentemente ao longo das épocas:
  - Época 1: ~1.0
  - Época 5: ~0.85
* Isto indica que o modelo estava aprendendo, mas ainda havia espaço para melhorias

## 5. Resultados e Performance

* **Acurácia Final**: 62.67% no conjunto de teste
* Esta performance é razoável para um modelo básico em CIFAR-10, considerando:
  - Arquitetura relativamente simples
  - Poucas épocas de treinamento
  - Batch size pequeno
  - Execução em CPU ao invés de GPU

## 6. Exportação do Modelo

* O modelo foi exportado para formato ONNX
* Benefícios do formato ONNX:
  - Interoperabilidade entre frameworks
  - Otimizações específicas para hardware
  - Compatibilidade com ferramentas de inferência

## 7. Importância da NVIDIA em Small Language Models

### Hardware Acceleration
* GPUs NVIDIA fornecem aceleração significativa para:
  - Treinamento de modelos
  - Inferência em produção
  - Otimização de performance

### Frameworks e Ferramentas
* **CUDA**: Framework para computação paralela
* **TensorRT**: Engine de inferência que otimiza modelos para hardware NVIDIA
* **Benefícios**:
  - Redução no tempo de treinamento
  - Menor latência na inferência
  - Melhor eficiência energética
  - Custos operacionais reduzidos

### Ecossistema para SLMs
* Suporte para quantização
* Otimização de memória
* Ferramentas de profiling e debugging
* Integração com frameworks populares

## 8. Oportunidades de Melhoria

1. **Hardware**:
   * Corrigir a configuração do CUDA para utilizar GPU
   * Aumentar o batch size com GPU disponível

2. **Modelo**:
   * Adicionar regularização (dropout, batch normalization)
   * Experimentar arquiteturas mais profundas
   * Implementar data augmentation

3. **Treinamento**:
   * Aumentar número de épocas
   * Implementar learning rate scheduling
   * Adicionar early stopping

4. **Otimização**:
   * Utilizar TensorRT para otimização do modelo
   * Implementar técnicas de quantização
   * Explorar pruning para redução do modelo

Este notebook demonstra o processo completo de desenvolvimento de um modelo de deep learning, desde a preparação dos dados até a exportação do modelo treinado, destacando a importância do ecossistema NVIDIA no desenvolvimento e otimização de Small Language Models.

Sobre o resultado: A acurácia de 62.67% no conjunto de teste significa que:

1. **Interpretação Básica**:
   * De todas as imagens no conjunto de teste, o modelo classificou corretamente 62.67% delas
   * Em outras palavras, para cada 100 imagens, aproximadamente 63 foram classificadas na categoria correta

2. **Contexto do CIFAR-10**:
   * Para o dataset CIFAR-10, que tem 10 classes diferentes, este resultado é:
     - Melhor que classificação aleatória (que seria 10%)
     - Razoável para um modelo básico sem otimizações
     - Abaixo do estado da arte (que alcança >95%)

3. **Fatores que Influenciaram este Resultado**:
   * Treinamento em CPU ao invés de GPU
   * Arquitetura relativamente simples
   * Apenas 5 épocas de treinamento
   * Batch size pequeno (4)
   * Sem técnicas de regularização ou data augmentation

4. **Comparação**:
   * Modelos mais sofisticados no CIFAR-10 alcançam:
     - ResNet: ~93-95%
     - DenseNet: ~94-96%
     - EfficientNet: ~98%

5. **Possíveis Melhorias**:
   * Aumentar o número de épocas
   * Implementar data augmentation
   * Usar arquitetura mais complexa
   * Adicionar regularização
   * Utilizar GPU para permitir mais experimentação

Este resultado serve como um bom baseline e demonstra que o modelo aprendeu padrões úteis, mas há bastante espaço para melhorias.